# A1)
## a)

$ \lVert Ax\rVert_1 = \sum\limits_{i=1}^{n} \lvert{{\sum\limits_{j=1}^{n} a_{ij}x_j}}\rvert $

$ \leq \sum\limits_{i=1}^{n} {\sum\limits_{j=1}^{n} | a_{ij} | | x_j} | = \sum\limits_{i=1}^{n} | x_j | {\sum\limits_{j=1}^{n} | a_{ij}} | $

$ \leq \sum\limits_{i=1}^{n} | x_j | {max}_{k=1,...,n} \sum\limits_{i=1}^{n} | a_{ik} | = \lVert x\rVert_1 {max}_{k=1,...,n} \sum\limits_{i=1}^{n} |a_{ik}| $

$ \implies \lVert A\rVert_1 \leq {max}_{k=1,...,n} \sum\limits_{i+1}^{n} |a_{ik}| \qquad\qquad\qquad\qquad\qquad\quad (1) $

Sei $ h^* $ die Spalte mit der max. Spaltensumme:

$ \implies x_0 = e_{h^*} $

$ \implies \lVert A_{x_{0}}\rVert_1 = max_{h=1,...,n} \sum\limits_{i=1}^{n} | a_{ik} | $

$ \implies \lVert A\rVert_1 \geq \frac{\lVert A_{x_{0}}\rVert_1}{\lVert x_0\rVert_1} = {max}_{l=1,...,n} \sum\limits_{i=1}^{n}  \qquad\qquad\qquad\qquad\qquad (2) $

Aus (1) und (2) $ \implies \lVert A\rVert_1 = {max}_{l=1,...,n} \sum\limits_{i=1}^{n} |a_{ij}| \qquad \square $

## b)

$ \lVert A_x\rVert_\infty = max_{i=1,...,n}\ |\sum\limits_{j=1}^{n}a_{ij}x_j | $

$ \leq max_{i=1,...,n} \sum\limits_{j=1}^{n}|a_{ij}|\ |x_j| $

$ \leq max_{i=1,...,n} \sum\limits_{j=1}^{n} |a_{ij}|\ \lVert x\rVert_\infty $

$ \implies \lVert A\rVert_\infty \leq max_{i=1,...,n} \sum\limits_{j=1}^{n} |a_{ij}| \qquad\qquad\qquad\qquad\qquad\quad (1) $

Sei $ x_0 = (1,1,...,1)^T $

$ \lVert A_{x_0}\rVert_\infty = max_{i=1,...,n} \sum\limits_{j=1}^{n} |a_{ij}{x_0}_j| = max_{i=1,...,n} \sum\limits_{j=1}^{n} |a_{ij}| \qquad\qquad (2) $

Aus (1) und (2) $ \implies \lVert A\rVert_\infty = max_{i=1,...,n} \sum\limits_{j=1}^{n} |a_{ij}| \qquad \square $

# A2)
## a)

$ \hat{A}=A+uv^T,\qquad u,v\in\mathbb{R}^n,\quad u\neq0,\quad v\neq0 $

z.Z.: $ \hat{A} $ ist genau dann singulaer, wenn $ 1+v^tA^{-1}u=0 $.

Idee: $ det(A+uv^T) = (1+v^TA^{-1}u)\ det(A) $

## c)

In [3]:
import numpy as np

def print_matrix(matrix): # utility
    s = [[str(e) for e in row] for row in matrix]
    lens = [max(map(len, col)) for col in zip(*s)]
    fmt = '\t'.join('{{:{}}}'.format(x) for x in lens)
    table = [fmt.format(*row) for row in s]
    print('\n'.join(table))

def zerlegung(A):
    n = len(A) # we assume A is a quadratic matrix
    p = [i for i in range(n-1)] # by default, we swap all n-1 rows/elements with themselves
    for c in range(n-1):
        for r in range(c+1, n):
            if A[c][c] == 0: # swap rows
                temp = A[c]
                A[c] = A[c+1]
                A[c+1] = temp
                p[c] = c+1
                #continue
            factor = (A[r][c] / A[c][c]) if A[c][c] != 0 else 0
            #print(f"Eliminating r={r}, c={c}, factor={factor}")
            for i in range(c, n):
                A[r][i] = A[r][i] - factor * A[c][i]
            A[r][c] = factor # inserting the associated L matrix value
    return A, p

def permutation(p, b):
    for i in range(len(p)):
        if p[i] == i:
            continue
        temp = b[p[i]]
        b[p[i]] = b[i]
        b[i] = temp
    return b

def vorwaerts(LU, c):
    n = len(c)
    y = []
    for row in range(n):
        y.append(c[row])
        for column in range(row):
            y[row] -= LU[row][column] * y[column]
    return y

def rueckwaerts(LU, y):
    n = len(y)
    x = [0 for i in range(n)]
    for row in reversed(range(n)):
        x[row] = y[row]
        for column in reversed(range(row+1, n)):
            x[row] -= LU[row][column] * x[column]
        x[row] /= LU[row][row]
    return x

def solve_for_x(A,b):
    """
    solve Ax=b with LU decomposition

    :param A: 2-D array representing a quadtratic matrix
    :param b: 1-D array representing a vector
    """

    LU,p=zerlegung(A)
    c=permutation(p,b)
    y=vorwaerts(LU,c)
    x=rueckwaerts(LU,y)
    print(f"y={y}, x={x}")

    return x

def sherman_morison(A,u,v,b):
    """
    solve Bx=b with B=A+uv^T using the sherman_morison algorithm

    :param A: 2-D array representing a regular matrix
    :param u: 1-D array representing a vector
    :param v: 1-D array representing a vector
    :param b: 1-D array representing a vector
    :return:
    """
    print("Az=u")
    z=solve_for_x(A,u)
    z=np.array(z)
    v=np.array(v)
    alpha=1+np.dot(v,z)
    if(alpha==0):
        print("B ist nicht regulär")
        return
    alpha=1/alpha
    print("Az_=b_")
    z_=solve_for_x(A,b)
    print("Bx=b")
    return z_-alpha*np.dot(v,z)*z

A = [[0,0,0,1],
     [2,1,2,0],
     [4,4,0,0],
     [2,3,1,0]]
b = [3, 5, 4, 5]
u=[0,1,2,3]
v=[0,0,0,1]

sherman_morison(A,u,v,b)

Az=u
y=[1, 0.0, 2.0, 0.0], x=[-0.8333333333333333, 1.3333333333333333, 0.6666666666666666, 0.0]
Az_=b_
y=[3, 2.0, 1.5, 5.0], x=[-0.7, 3.8, 0.3, 5.0]
Bx=b


array([-0.7,  3.8,  0.3,  5. ])

# A3)

$ f(x_1,x_2) = \frac{x_1}{x_2},\quad x_1=1,\ x_2=10^{-4} $

Using differential error analysis to approximate $ \varkappa_a$.

$ f'(x_1, x_2) = \begin{pmatrix}
\frac{1}{x_2} & -\frac{x_1}{x_2^2} \\
\end{pmatrix}\ $

ehhehe REDACTED ehehe